In [9]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2
pd.options.mode.chained_assignment = None

In [2]:
# alc_tob_caf use-case
# articles = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

# articles = articles[articles.topics.apply(lambda x: (max(eval(x)[0]) != 0.062500) & (max(eval(x)[0]) == eval(x)[0][11]))]
# articles = articles[articles.entities.apply(lambda x: any(key in x for key in ['alcohol', 'tobacco', 'caffeine']))]

# articles.to_csv(cache_dir+'alc_tob_caf.tsv', sep='\t', index=None)

# aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v2.tsv', cache_dir+'alc_tob_caf.tsv', cache_dir+'alc_tob_caf.tsv')

# articles = pd.read_csv(cache_dir+'alc_tob_caf.tsv', sep='\t')
# articles = articles[articles.replies.apply(lambda x: x != '[]')][['url', 'title']]

# df1 = pd.read_csv(cache_dir+'similarity_model/train_pairs_v1.tsv', sep='\t')
# df1 = df1[df1['related']]
# df2 = pd.read_csv(cache_dir+'similarity_model/test_pairs_v3.tsv', sep='\t')
# df2 = df2[df2['related']]
# df = pd.concat([df1, df2]).drop('related', axis=1)

# articles.merge(df, left_on='url', right_on='article').drop('url', axis=1).groupby(['article', 'title'])['paper'].apply(lambda x: '\n'.join(x)).reset_index().to_csv(cache_dir+'alc_tob_caf.tsv', sep='\t')

# crispr use-case
# articles = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

# articles = articles[articles.entities.apply(lambda x: any(key in x for key in ['CRISPR']))]
# articles = articles[articles['url'].apply(lambda x: not x.startswith('http://www.coach-boursier.com'))]

# articles.to_csv(cache_dir+'crispr.tsv', sep='\t', index=None)

# aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v2.tsv', cache_dir+'crispr.tsv', cache_dir+'crispr.tsv')

# articles = pd.read_csv(cache_dir+'crispr.tsv', sep='\t')

# df1 = pd.read_csv(cache_dir+'similarity_model/train_pairs_v1.tsv', sep='\t')
# df1 = df1[df1['related']]
# df2 = pd.read_csv(cache_dir+'similarity_model/test_pairs_v3.tsv', sep='\t')
# df2 = df2[df2['related']]
# df = pd.concat([df1, df2]).drop('related', axis=1)

# articles.merge(df, left_on='url', right_on='article').drop('url', axis=1).groupby(['article', 'title'])['paper'].apply(lambda x: '\n'.join(x)).reset_index().to_csv(cache_dir+'crispr.tsv', sep='\t')